In [13]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import folium

from folium import FeatureGroup, Map, Marker
import pandas as pd
import numpy as np

In [14]:
df = pd.read_feather(r"D:\Work for MWG\Dash folium\Data/Data.feather")
df = df[df['Date'] >= datetime(2023, 7, 1).date()]
df.head(2)

,Date,Mã sản phẩm,Mã siêu thị,Số lượng hủy NCC,Số lượng nhập,Số lượng xuất giảm giá,Số lượng xuất hủy,Số lượng xuất nguyên giá
1573283,2023-07-01,1232844000391,161,0.0,5.6962,0.0,0.0,1.098
1573284,2023-07-01,1232844000391,191,0.0,35.6010,0.0,0.0,7.686


In [15]:
df = pd.pivot_table(df,index="Mã siêu thị",values="Số lượng xuất nguyên giá",aggfunc=np.sum).reset_index()

In [16]:
df_store = pd.read_feather(r"D:\Work for MWG\Dash folium\Data\GetStoreDistance.feather")
df_store.head(2)

,Mã siêu thị,Kho đến,Kinh độ,Vĩ độ
0,6511,6511 - BHX_DLA_EKA - Thôn 8 Ea Ô,12.708583,-108.4860
1,6114,6114 - BHX_DLA_KPA - 501 Giải Phóng,12.713417,-108.2905


In [17]:
dataset = pd.merge(left=df,right=df_store,on=['Mã siêu thị'],how='left')
dataset = dataset.dropna(axis=0)
dataset.head()

,Mã siêu thị,Số lượng xuất nguyên giá,Kho đến,Kinh độ,Vĩ độ
0,116,826.660511,116 - BHX_HCM_TPH - 195 Dương Văn Dương,10.792515,106.617240
1,161,983.096000,161 - BHX_HCM_BTA - 20 Trương Phước Phan,10.767924,106.613604
2,191,3482.197593,191 - BHX_HCM_BTA - 393 Hương Lộ 3,10.801733,106.610570
3,202,771.626932,202 - BHX_HCM_BTA - 473/4A Tỉnh Lộ 10,10.756792,106.619120
4,1157,1716.736448,1157 - BHX_HCM_BTA - 175 Lê Đình Cẩn,10.759498,106.598318


In [20]:



lat = 10.8231
long = 106.6297
zoom = 12
gmap2 = folium.Map(location=(lat, long), zoom_start=zoom)

def _addMarker(gmap):
    for i, row in dataset.iterrows():
        # Tính bán kính dựa trên số lượng bán
        radius = row['Số lượng xuất nguyên giá'] / 500
        # Tạo đánh dấu với bán kính và màu sắc tương ứng
        marker = folium.CircleMarker(location=(row['Kinh độ'], row['Vĩ độ']), radius=radius, color='red', fill=True, fill_color='red', fill_opacity=0.5, popup=row['Kho đến'])
        marker.add_to(gmap)

In [21]:
_addMarker(gmap2)
gmap2